In [30]:
import sys
sys.path.append('..')

from services.adapter.ml_utils import RabbitWrapper

In [31]:
redash = RabbitWrapper('../configs/resources.ini', 'adapter_local', swap_topics=True)

2024-09-29 05:44:16.855 | INFO     | services.adapter.ml_utils.brokers.rabbit.wrapper:_load_config:108 - Config has been loaded
2024-09-29 05:44:16.857 | INFO     | services.adapter.ml_utils.brokers.rabbit.wrapper:_connect:131 - Trying to connect at 1 time
2024-09-29 05:44:16.863 | INFO     | services.adapter.ml_utils.brokers.rabbit.wrapper:_connect:141 - Connection successful
2024-09-29 05:44:16.865 | INFO     | services.adapter.ml_utils.brokers.rabbit.wrapper:__init__:84 - Input topic piracy_detection_output has been connected
2024-09-29 05:44:16.866 | INFO     | services.adapter.ml_utils.brokers.rabbit.wrapper:__init__:88 - Output topic piracy_detection_input has been connected


In [32]:
msgs = redash.listen()

2024-09-29 05:44:17.813 | INFO     | services.adapter.ml_utils.brokers.rabbit.wrapper:listen:203 - Start consuming on piracy_detection_output


In [33]:
len(msgs)

1000

In [34]:
d = {i['inputs']['video_link'].split('.')[0].split('/')[-1]: i['result']['duplicate_for'].split('.')[0].split('/')[-1] for i in msgs if i['result']['duplicate_for']}

In [22]:
len(d)

244

In [35]:
import pandas as pd
data = pd.read_csv('test.csv')
# data = data[:2000]
data

,created,uuid,link
0,2024-06-01 01:37:57,49577a11-51b9-490a-b1f0-df17335219de,https://s3.ritm.media/yappy-db-duplicates/4957...
1,2024-06-01 04:42:10,4e1f7fad-5008-4216-9849-550a00f1e35f,https://s3.ritm.media/yappy-db-duplicates/4e1f...
2,2024-06-01 08:44:48,337fdbe6-2bc7-4bc7-931e-d94ada927ede,https://s3.ritm.media/yappy-db-duplicates/337f...
3,2024-06-01 10:11:48,35138a88-0249-405e-91b4-8a36b1e2e730,https://s3.ritm.media/yappy-db-duplicates/3513...
4,2024-06-01 12:23:29,322f4312-3d46-401b-8cd9-80a0d06347ed,https://s3.ritm.media/yappy-db-duplicates/322f...
...,...,...,...
995,2024-09-07 17:10:54,933149f9-e660-4377-95c8-f8dd329db24e,https://s3.ritm.media/yappy-db-duplicates/9331...
996,2024-09-08 05:18:24,9f707190-3b32-48bf-a5f4-ceec6eedb847,https://s3.ritm.media/yappy-db-duplicates/9f70...
997,2024-09-10 04:45:04,e0fe3e65-8aa6-4f38-bb83-85948d5539bc,https://s3.ritm.media/yappy-db-duplicates/e0fe...
998,2024-09-11 04:42:01,ef7e175e-2391-45a0-b69f-33837668bb79,https://s3.ritm.media/yappy-db-duplicates/ef7e...


In [45]:
import numpy as np
rows = []

tp = 0
fp = 0
fn = 0
tn = 0

is_duplicate = []
duplicates_for = []

for i, row in data.iterrows():
    is_dupicate = False
    duplicate_for = None
    
    if row['uuid'] in set(d.keys()):
        duplicate_for = d[row['uuid']]
        is_dupicate = True
    
    is_duplicate.append(is_dupicate)
    duplicates_for.append(duplicate_for)
    
            

In [46]:
data['is_duplicate'] = is_duplicate
data['duplicate_for'] = duplicates_for

In [47]:
data

,created,uuid,link,is_duplicate,duplicate_for
0,2024-06-01 01:37:57,49577a11-51b9-490a-b1f0-df17335219de,https://s3.ritm.media/yappy-db-duplicates/4957...,False,None
1,2024-06-01 04:42:10,4e1f7fad-5008-4216-9849-550a00f1e35f,https://s3.ritm.media/yappy-db-duplicates/4e1f...,False,None
2,2024-06-01 08:44:48,337fdbe6-2bc7-4bc7-931e-d94ada927ede,https://s3.ritm.media/yappy-db-duplicates/337f...,False,None
3,2024-06-01 10:11:48,35138a88-0249-405e-91b4-8a36b1e2e730,https://s3.ritm.media/yappy-db-duplicates/3513...,False,None
4,2024-06-01 12:23:29,322f4312-3d46-401b-8cd9-80a0d06347ed,https://s3.ritm.media/yappy-db-duplicates/322f...,False,None
...,...,...,...,...,...
995,2024-09-07 17:10:54,933149f9-e660-4377-95c8-f8dd329db24e,https://s3.ritm.media/yappy-db-duplicates/9331...,True,06e0d485-d4e1-4ba7-8eb2-e2f1fcfca416
996,2024-09-08 05:18:24,9f707190-3b32-48bf-a5f4-ceec6eedb847,https://s3.ritm.media/yappy-db-duplicates/9f70...,True,56cfa0d5-a050-4443-9a80-1403bcad6839
997,2024-09-10 04:45:04,e0fe3e65-8aa6-4f38-bb83-85948d5539bc,https://s3.ritm.media/yappy-db-duplicates/e0fe...,False,None
998,2024-09-11 04:42:01,ef7e175e-2391-45a0-b69f-33837668bb79,https://s3.ritm.media/yappy-db-duplicates/ef7e...,True,0574335c-9884-40de-a514-97b3fd3d72df


In [29]:
data.to_csv('submission_8frames_0.7v_0.1a.csv', index=False)